In [1]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import os
from dash import Dash
from dash import Dash, dcc, html, Input, Output


/Users/vishalsehgal/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Main Feature
1. Players performance coomparision : Compare players from different clubs when they face each other in their national teams during the World Cup.
2. Team performance comparision : Analyze team performance and how well players know each other's strengths and weaknesses.


Steps:
1. Integrate Match Data:    Use the Fifa_Worldcup_2022_Groups.csv for match data.
2. Filter Players Based on Match Selection:     Allow users to filter players by match, country, and club.
3. Visual Comparison:   Display a visual comparison of player performance.

In [2]:
player_data_file = 'assets/data/Players_Data_2022/FIFA22_official_data.csv'
match_data_file = 'assets/data/Matchdata.csv'
player_df = pd.read_csv(player_data_file)
match_df = pd.read_csv(match_data_file)
match_df.columns.tolist()
#match_df.head()

['match',
 'dayofweek',
 'match_time',
 'home_team',
 'away_team',
 'home_xg',
 'away_xg',
 'score',
 'attendance',
 'venue',
 'referee',
 'home_formation',
 'away_formation',
 'home_captain',
 'away_captain',
 'home_manager',
 'away_manager',
 'home_possession',
 'away_possession',
 'home_completed_passes',
 'home_attempted_pases',
 'away_completed_passes',
 'away_attempted_pases',
 'home_sot',
 'away_sot',
 'home_total_shots',
 'away_total_shots',
 'home_saves',
 'away_saves',
 'home_fouls',
 'away_fouls',
 'home_corners',
 'away_corners',
 'home_crosses',
 'away_crosses',
 'home_touches',
 'away_touches',
 'home_tackles',
 'away_tackles',
 'home_interceptions',
 'away_interceptions',
 'home_aerials_won',
 'away_aerials_won',
 'home_clearances',
 'away_clearances',
 'home_offsides',
 'away_offsides',
 'home_gks',
 'away_gks',
 'home_throw_ins',
 'away_throw_ins',
 'home_long_balls',
 'away_long_balls']

In [3]:
player_stats = player_df[['Name', 'Nationality', 'Overall', 'Club', 
                          'Crossing', 'Finishing', 'HeadingAccuracy', 
                          'ShortPassing', 'Volleys', 'Dribbling', 
                          'Curve', 'FKAccuracy', 'LongPassing', 
                          'BallControl', 'Acceleration', 'SprintSpeed', 
                          'Agility', 'Reactions', 'Balance', 'ShotPower', 
                          'Jumping', 'Stamina', 'Strength', 'LongShots', 
                          'Aggression', 'Interceptions', 'Positioning', 
                          'Vision', 'Penalties', 'Composure']]

unique_matches = match_df.apply(lambda x: f"{x['home_team']} vs {x['away_team']}", axis=1).unique()
unique_players = player_stats['Name'].unique()


In [4]:
# Initialize the Dash app
app = JupyterDash(__name__)

/Users/vishalsehgal/Library/Python/3.9/lib/python/site-packages/dash/dash.py:556: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [5]:
# Define the layout of the Dash app
app.layout = html.Div([
    html.Header([
        html.Img(src=app.get_asset_url('Worldcuplogo.png')),
        html.Div([
            html.H1('FIFA COMPARISON', className='header-title'),
            html.H2('Exploring players play against each other in different teams', className='header-content'),
            html.H3('Dashboard created by: Vishal Sehgal and Hieke van Heesch', className='header-content')
        ], className='header-content')
    ], className='header'),
    html.Div([
        dcc.Dropdown(
            id='match-dropdown',
            options=[{'label': match, 'value': match} for match in unique_matches],
            value=None,
            placeholder="Select Match"
        ),
        dcc.Dropdown(
            id='player1-dropdown',
            options=[],
            value=None,
            placeholder="Select Player 1"
        ),
        dcc.Dropdown(
            id='player2-dropdown',
            options=[],
            value=None,
            placeholder="Select Player 2"
        ),
    ], className='dropdown-container'),
    dcc.Graph(id='comparison-graph')
])

In [6]:
# Define the callback to update player dropdowns based on selected match
@app.callback(
    [Output('player1-dropdown', 'options'),
     Output('player2-dropdown', 'options')],
    [Input('match-dropdown', 'value')]
)
def update_player_dropdowns(selected_match):
    if selected_match is None:
        return [], []

    home_team, away_team = selected_match.split(' vs ')
    players_in_match = player_stats[player_stats['Nationality'].isin([home_team, away_team])]
    player_options = [{'label': player, 'value': player} for player in players_in_match['Name'].unique()]
    
    return player_options, player_options

# Define the callback to update the graph
@app.callback(
    Output('comparison-graph', 'figure'),
    [Input('player1-dropdown', 'value'),
     Input('player2-dropdown', 'value')]
)
def update_graph(player1, player2):
    if player1 is None or player2 is None:
        return go.Figure()

    player1_stats = player_stats[player_stats['Name'] == player1].iloc[0]
    player2_stats = player_stats[player_stats['Name'] == player2].iloc[0]

    categories = player1_stats.index[4:]  # Performance metrics columns
    player1_values = player1_stats[4:].values
    player2_values = player2_stats[4:].values

    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=player1_values,
        theta=categories,
        fill='toself',
        name=player1
    ))

    fig.add_trace(go.Scatterpolar(
        r=player2_values,
        theta=categories,
        fill='toself',
        name=player2
    ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )
        ),
        showlegend=True,
        title=f'Comparison: {player1} vs {player2}'
    )

    return fig

# Run the app
app.run_server(mode='inline')
